In [1]:
import json
from datetime import datetime, timedelta

def generate_exam_data():
    # Define colleges and departments
    colleges = {
        "SMS": ["Economics", "Accounting", "Business Administration"],
        "NAS": ["Biology", "Physics", "Mathematics", "Microbiology"],
        "CIS": ["Software Engineering", "Computer Science", "Cybersecurity"],
    }

    departments = [
        {"department_id": "ECON", "department_name": "Economics", "college_id": "SMS"},
        {"department_id": "ACCT", "department_name": "Accounting", "college_id": "SMS"},
        {"department_id": "BIO", "department_name": "Biology", "college_id": "NAS"}
    ]

    # Define regular and combined courses
    courses = [
        {"course_id": "ECO101", "course_name": "Introduction to Economics", "level_id": 1},
        {"course_id": "ACC102", "course_name": "Financial Accounting", "level_id": 2}
    ]

    combined_courses = [
        {
            "course_id": "GSP101",
            "course_name": "Use of English (CBT)",
            "departments": ["ECON", "ACCT", "BIO"]
        }
    ]

    # Define levels and student numbers
    levels = [
        {"level_id": 1, "department_id": "ECON", "num_students": 120},
        {"level_id": 2, "department_id": "ACCT", "num_students": 100},
        {"level_id": 1, "department_id": "BIO", "num_students": 90}
    ]

    # Generate time slots dynamically
    start_time = datetime.strptime("08:00", "%H:%M")
    end_time = datetime.strptime("18:00", "%H:%M")
    break_time = datetime.strptime("12:00", "%H:%M")

    time_slots = []
    current_time = start_time
    while current_time < end_time:
        next_time = current_time + timedelta(hours=2)
        if current_time == break_time:
            current_time += timedelta(hours=1)  # Skip the break
            continue
        time_slots.append(
            f"{current_time.strftime('%H:%M')} - {next_time.strftime('%H:%M')}"
        )
        current_time = next_time

    # Generate dynamic date range with actual dates and days
    start_date = datetime.strptime("2024-01-08", "%Y-%m-%d")  # Monday
    end_date = datetime.strptime("2024-01-21", "%Y-%m-%d")  # 2 weeks later

    date_range = []
    current_date = start_date
    while current_date <= end_date:
        day_name = current_date.strftime("%A")
        date_range.append(
            {"date": current_date.strftime("%Y-%m-%d"), "day": day_name}
        )
        current_date += timedelta(days=1)

    # Combine all data into one dictionary
    data = {
        "colleges": colleges,
        "departments": departments,
        "courses": courses,
        "combined_courses": combined_courses,
        "levels": levels,
        "time_slots": time_slots,
        "date_range": date_range,
    }

    # Save to JSON
    with open("data5.json", "w") as file:
        json.dump(data, file, indent=4)

    print("Exam data generated and saved to 'data5.json'.")

# Run the generator
generate_exam_data()


Exam data generated and saved to 'data5.json'.


In [3]:
import json
from collections import defaultdict

def generate_exam_timetable(courses, combined_courses, levels, time_slots, date_range):
    """
    Generates an exam timetable based on courses, combined courses, levels, and time slots.
    Ensures no overlap of exams for departments, levels, and combined courses.
    """
    timetable = defaultdict(list)  # Timetable as a dictionary with dates as keys
    used_slots = defaultdict(set)  # Track used slots for levels and combined courses

    current_date_index = 0
    current_time_slot_index = 0

    for course in courses:
        course_id = course["course_id"]
        course_name = course["course_name"]
        level_id = course["level_id"]

        # Find the department and number of students for the level
        level_data = next(
            (level for level in levels if level["level_id"] == level_id), None
        )
        if not level_data:
            continue  # Skip if level data is not found

        department_id = level_data["department_id"]
        num_students = level_data["num_students"]

        # Schedule the course
        scheduled = False
        while not scheduled:
            current_date = date_range[current_date_index]
            date = current_date["date"]
            day = current_date["day"]

            # Ensure exams are scheduled only on weekdays or Saturdays for overflow
            if day in ["Saturday", "Sunday"] and current_date_index < len(date_range) - 1:
                current_date_index += 1
                continue

            # Get current time slot
            current_time_slot = time_slots[current_time_slot_index]

            # Check if the time slot is available for the department and level
            if (department_id, level_id) not in used_slots[date]:
                timetable[date].append(
                    {
                        "day": day,
                        "time": current_time_slot,
                        "course": {"id": course_id, "name": course_name},
                        "programs": [department_id],
                    }
                )
                # Mark this time slot as used
                used_slots[date].add((department_id, level_id))
                scheduled = True
            else:
                # Move to the next time slot or date
                current_time_slot_index += 1
                if current_time_slot_index >= len(time_slots):
                    current_time_slot_index = 0
                    current_date_index += 1

    # Schedule combined courses
    for combined_course in combined_courses:
        course_id = combined_course["course_id"]
        course_name = combined_course["course_name"]
        departments = combined_course["departments"]

        scheduled = False
        while not scheduled:
            current_date = date_range[current_date_index]
            date = current_date["date"]
            day = current_date["day"]

            # Ensure exams are scheduled only on weekdays or Saturdays for overflow
            if day in ["Saturday", "Sunday"] and current_date_index < len(date_range) - 1:
                current_date_index += 1
                continue

            # Get current time slot
            current_time_slot = time_slots[current_time_slot_index]

            # Check if the time slot is available for all departments
            overlap = any(
                (dept_id, level_id) in used_slots[date]
                for dept_id in departments
                for level_id in [level["level_id"] for level in levels if level["department_id"] == dept_id]
            )
            if not overlap:
                timetable[date].append(
                    {
                        "day": day,
                        "time": current_time_slot,
                        "course": {"id": course_id, "name": course_name},
                        "programs": departments,
                    }
                )
                # Mark this time slot as used for all departments
                for dept_id in departments:
                    for level in levels:
                        if level["department_id"] == dept_id:
                            used_slots[date].add((dept_id, level["level_id"]))
                scheduled = True
            else:
                # Move to the next time slot or date
                current_time_slot_index += 1
                if current_time_slot_index >= len(time_slots):
                    current_time_slot_index = 0
                    current_date_index += 1

    return timetable


def export_exam_timetable():
    """
    Exports the generated exam timetable to a JSON file.
    """
    with open("data5.json", "r") as file:
        data = json.load(file)

    courses = data["courses"]
    combined_courses = data["combined_courses"]
    levels = data["levels"]
    time_slots = data["time_slots"]
    date_range = data["date_range"]

    timetable = generate_exam_timetable(courses, combined_courses, levels, time_slots, date_range)

    with open("timetable5.json", "w") as file:
        json.dump(timetable, file, indent=4)

    print("Exam timetable exported to 'timetable5.json'.")


# Run the timetable generator
export_exam_timetable()


Exam timetable exported to 'timetable5.json'.
